In [1]:
import os
import glob
import configparser
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col
pd.set_option('max_colwidth',800)

In [2]:
config = configparser.ConfigParser()

config.read_file(open('dl.cfg'))

os.environ["AWS_ACCESS_KEY_ID"]= config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"]= config['AWS']['AWS_SECRET_ACCESS_KEY']

### Create Spark Session

In [3]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

In [4]:
def get_file(filepath):
    all_file=[]
    for root,dirs,files in os.walk(filepath):
        files=glob.glob(os.path.join(root,"*json"))
        for f in files:
            all_file.append(os.path.abspath(f))
    return all_file

In [7]:
# load data 
sample_song = spark.read.json(get_file('data/song_data')[0])
sample_log = spark.read.json(get_file('data/log-data')[0])

In [ ]:
get_file()

In [29]:
# check the data 
sample_song.printSchema()
sample_song.toPandas()


root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR7G5I41187FB4CE6C,None,"London, England",None,Adam Ant,233.40363,1,SONHOTT12A8C13493C,Something Girls,1982


In [30]:
sample_log.printSchema()
sample_log.limit(5).toPandas()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",32


## Process `song_data`

### #1 Songs Table

In [110]:
# table result
songs_table=sample_song.select(['song_id','title','artist_id','year','duration'])

In [113]:
songs_table.write.partitionBy('year', 'artist_id').parquet(os.path.join('data/', 'songs.parquet'), 'overwrite')

### #2 artist Table

In [32]:
# table result
sample_song.select(['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']).show()

+------------------+-----------+---------------+---------------+----------------+
|         artist_id|artist_name|artist_location|artist_latitude|artist_longitude|
+------------------+-----------+---------------+---------------+----------------+
|AR7G5I41187FB4CE6C|   Adam Ant|London, England|           null|            null|
+------------------+-----------+---------------+---------------+----------------+



## Process `log_data`

### #3 time Table

In [47]:
# convert ts to time stamp
sample_log=sample_log.withColumn('ts', F.from_unixtime((F.col('ts')/1000)))
sample_log.select(['ts']).limit(5).toPandas()

,ts
0,2018-11-10 18:33:56
1,2018-11-10 18:36:10
2,2018-11-10 18:40:34
3,2018-11-10 20:34:01
4,2018-11-10 20:36:13


In [57]:
sample_log=sample_log.withColumn('month', F.month('ts'))
sample_log=sample_log.withColumn('year', F.year('ts'))
sample_log=sample_log.withColumn('day', F.dayofyear('ts'))
sample_log=sample_log.withColumn('hour', F.hour('ts'))
sample_log=sample_log.withColumn('weekday', F.dayofweek('ts'))
sample_log=sample_log.withColumn('week', F.weekofyear('ts'))

In [60]:
sample_log.select(['ts','month','year','day','hour','weekday','week']).limit(10).toPandas()

,ts,month,year,day,hour,weekday,week
0,2018-11-10 18:33:56,11,2018,314,18,7,45
1,2018-11-10 18:36:10,11,2018,314,18,7,45
2,2018-11-10 18:40:34,11,2018,314,18,7,45
3,2018-11-10 20:34:01,11,2018,314,20,7,45
4,2018-11-10 20:36:13,11,2018,314,20,7,45
5,2018-11-10 20:36:46,11,2018,314,20,7,45
6,2018-11-11 01:56:28,11,2018,315,1,1,45
7,2018-11-11 02:00:47,11,2018,315,2,1,45
8,2018-11-11 02:04:07,11,2018,315,2,1,45
9,2018-11-11 02:08:28,11,2018,315,2,1,45


In [116]:
sample_log.selectExpr('ts as start_time').limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,status,ts,userAgent,userId,month,year,day,hour,weekday,week
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,...,200,2018-11-10 18:33:56,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",69,11,2018,314,18,7,45
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,...,200,2018-11-10 18:36:10,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",69,11,2018,314,18,7,45
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,...,200,2018-11-10 18:40:34,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",69,11,2018,314,18,7,45
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,...,200,2018-11-10 20:34:01,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",32,11,2018,314,20,7,45
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,...,200,2018-11-10 20:36:13,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",32,11,2018,314,20,7,45


In [61]:
# table result
sample_log.select(['ts','hour','day','week','month','year','weekday']).limit(10).toPandas()

,ts,hour,day,week,month,year,weekday
0,2018-11-10 18:33:56,18,314,45,11,2018,7
1,2018-11-10 18:36:10,18,314,45,11,2018,7
2,2018-11-10 18:40:34,18,314,45,11,2018,7
3,2018-11-10 20:34:01,20,314,45,11,2018,7
4,2018-11-10 20:36:13,20,314,45,11,2018,7
5,2018-11-10 20:36:46,20,314,45,11,2018,7
6,2018-11-11 01:56:28,1,315,45,11,2018,1
7,2018-11-11 02:00:47,2,315,45,11,2018,1
8,2018-11-11 02:04:07,2,315,45,11,2018,1
9,2018-11-11 02:08:28,2,315,45,11,2018,1


### #4 users table

In [62]:
sample_log.select(['userId','firstName','lastName','gender','level']).limit(5).toPandas()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
3,32,Lily,Burns,F,free
4,32,Lily,Burns,F,free


### #5 songplays table

In [95]:
sample_log_nextSong=sample_log.filter(sample_log.page=='NextSong').\
                        select(['ts', 'userId', 'level', 'song', 'artist',
                                'sessionId', 'location', 'userAgent'])

In [99]:
sample_song.limit(10).toPandas()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR7G5I41187FB4CE6C,None,"London, England",None,Adam Ant,233.40363,1,SONHOTT12A8C13493C,Something Girls,1982


In [100]:
# join table
joined_df=sample_song.join(sample_log_nextSong,
                           sample_song.title==sample_log_nextSong.song)


In [9]:
sample_log.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",32
